# Package Installation

In [1]:
pip install gradio

     |████████████████████████████████| 2.9 MB 27.8 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 2.0 MB 9.7 MB/s 
     |████████████████████████████████| 253 kB 18.9 MB/s 
     |████████████████████████████████| 211 kB 40.0 MB/s 
     |████████████████████████████████| 1.1 MB 34.8 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 84 kB 2.3 MB/s 
     |████████████████████████████████| 271 kB 34.1 MB/s 
     |████████████████████████████████| 144 kB 42.3 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
     |████████████████████████████████| 10.9 MB 45.3 MB/s 
     |████████████████████████████████| 79 kB 4.2 MB/s 
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 61 kB 240 kB/s 
     |████████████████████████████████| 856 kB 44.0 MB/s 
     |████████████████████████

# Import Packages

In [2]:
import numpy as np
import pandas as pd
import gradio as gr
import warnings
from pathlib import Path
import tensorflow as tf
import os.path
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
# Helper libraries
import matplotlib.pyplot as pl
warnings.filterwarnings('ignore')

# Google Drive

In [3]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/fruit-and-vegetable-image-recognition'

# Get the absolute path of the train folder
abspath_train = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/train/'

# Get the absolute path of the test folder
abspath_test = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/test/'

# Get the absolute path of the test folder
abspath_validation = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/validation/'

Mounted at /content/drive


# Import data

## Import FDC data

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf

protein = pd.read_csv('/content/drive/My Drive/NLP_food_protein.csv')
protein_df = pd.DataFrame(protein)

nutrient = pd.read_csv('/content/drive/My Drive/NLP_food_nutrient.csv')
nutrient_df = pd.DataFrame(nutrient)

## Import image data

In [5]:


# Create a list with the filepaths for training and testing
train_dir = Path(abspath_train)
train_filepaths = list(train_dir.glob(r'**/*.jpg'))

test_dir = Path(abspath_test)
test_filepaths = list(test_dir.glob(r'**/*.jpg'))

val_dir = Path(abspath_validation)
val_filepaths = list(test_dir.glob(r'**/*.jpg'))

def proc_img(filepath):
    """ Create a DataFrame with the filepath and the labels of the pictures
    """

    labels = [str(filepath[i]).split("/")[-2] \
              for i in range(len(filepath))]

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
    df = pd.concat([filepath, labels], axis=1)

    # Shuffle the DataFrame and reset index
    df = df.sample(frac=1).reset_index(drop = True)
    
    return df

train_df = proc_img(train_filepaths)
test_df = proc_img(test_filepaths)
val_df = proc_img(val_filepaths)

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_images = train_generator.flow_from_dataframe(
    dataframe=val_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 2780 validated image filenames belonging to 36 classes.
Found 334 validated image filenames belonging to 36 classes.
Found 334 validated image filenames belonging to 36 classes.


# Loading Model

In [7]:
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D
from keras.layers.core import Dense
model = load_model('/content/drive/My Drive/DATS6501/Mobilenet_model/fullmodel.h5')
x = model.output


# Functions

## Provide info based on input from FDC data


### Information extraction from nutrient data

In [8]:
def nutrientcheck(fdcid):
  stringlist = ""
  foodlist = nutrient_df[nutrient_df['fdc_id'] == fdcid]
  i = 0
  nutrient = []
  while i < len(foodlist):
      item = foodlist.iloc[i,]
      nutrientname = item['name']
      if nutrientname not in nutrient:
        nutrient.append(item['name'])
        nutrientamount = item['amount']
        unit = item['unit_name']
        singlestring = """{}: {}{}.\n""".format(nutrientname, nutrientamount, unit)
        stringlist = stringlist + singlestring
      i += 1
  print(nutrient)
  return stringlist

### Information extraction from protein data

In [9]:
def proteincheck(input):
  totalstring = ""
  for food in input:
    stringlist = """Information for {} products: \n\n""".format(food)
    foodlist = protein_df.loc[protein_df['food_name'].str.contains(food, case=False)]
    i = 0
    while i < len(foodlist):
      item = foodlist.iloc[i,]
      des = item['description']
      proteinval = item['protein_value'] * 4
      fatval = item['fat_value'] * 9
      carbonval = item['carbohydrate_value'] * 4
      total = proteinval + fatval + carbonval
      fdcid = item['fdc_id']
      singlestring = """Food Name: {}.\nTotal Calories: {}Kcal.\nProteins: {}Kcal.\nFat: {}Kcal.\nCarbonhydrate: {}Kcal.\n""".format(des, total, proteinval, fatval, carbonval)
      nutrientstring = nutrientcheck(fdcid)
      stringlist = stringlist + singlestring + nutrientstring + "\n"
      i += 1
    totalstring = totalstring + stringlist
  return totalstring


## Load Images

In [10]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image)
   np_image = transform.resize(np_image, (224, 224))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

## Predict using model

In [11]:


def imagemodel(imagepath):
  imagep = load(imagepath)
  # Predict the label of the test_images
  pred = model.predict(imagep)
  pred = np.argmax(pred,axis=1)
  # Map the label
  labels = (train_images.class_indices)
  labels = dict((v,k) for k,v in labels.items())
  pred = [labels[k] for k in pred]
  return pred


## Picture Recognition Function

In [12]:
def foodimage(imagepath):
  model_out = imagemodel(imagepath)
  foodinfo = proteincheck(model_out)
  return foodinfo
  


# Create Gradio Interface

In [13]:
outputs = gr.outputs.Textbox()

app = gr.Interface(fn=foodimage, inputs=['text'], outputs=outputs,description="Food Information Extraction (Image Recognition)")

# Launch the Gradio Web App

In [14]:
imagepath = '/content/drive/My Drive/fruit-and-vegetable-image-recognition/test/garlic/Image_7.jpg'
app.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://46325.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f8551906190>,
 'http://127.0.0.1:7860/',
 'https://46325.gradio.app')